In [1]:
import time
import math
import nltk
from textblob import TextBlob
import string
import pandas as pd
from nltk.corpus import stopwords
import itertools
from collections import Counter
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
from afinn import Afinn
from scipy import sparse
import numpy as np
from scipy.sparse import csr_matrix
import random

In [2]:
def lowletter(text):
    cleantext = text.lower()
    return cleantext

def cleansymbols(text):
    afterclean=''
    valid_letters = string.ascii_letters
    for i in text:
        if i in valid_letters  or i == " " and not (i.isdigit()):
            afterclean += i
    return afterclean

def cleanstopwords(text,stopWords) :
    tokens = nltk.word_tokenize(text)
    cleanstopwords = ""
    for word in tokens:
        if word not in stopWords and len(word)>1:
            cleanstopwords += word
            cleanstopwords += " "
    return cleanstopwords

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []


def get_avg_time(recipes,data,user_id):
    sem2=data.loc[data['user_id']== user_id]
    s=recipes.loc[recipes['id'].isin(sem2['recipe_id'])]
    return s['minutes'].mean()


        

def get_recommendations(user_id,name,df, cosine_sim):
    # Get the index of the recipe that matches the name
    indices = pd.Series(df.index, index=df['name']).drop_duplicates()
    
    idx = indices[name]

    # Get the pairwsie similarity scores of all recipes with that recipe
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the recipes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar recipes
    sim_scores = sim_scores[1:10]

    time=get_avg_time(recipes,sem,user_id)
    
    recipe_indices = [i[0] for i in sim_scores]

    df1=df.iloc[recipe_indices]
    df1=df1[df1['minutes']> (time-20)]
    df1=df1[df1['minutes']< (time+20)]

    # Return the top 10 most similar recipes
    return list(df1['name'])  

# Python program to convert a list to string
	
# Function to convert
def listToString(s):
    listToStr = ' '.join([str(elem) for elem in s])
    return listToStr

In [3]:
def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing 3 columns (userId, recipe_id, rating)

    Returns:
        X: sparse matrix
        user_mapper: dict that mapsto user user id's  indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps recipe id's to recipe indices
        movie_inv_mapper: dict that maps recipe indices to recipe id's
    """
    M = df['user_id'].nunique()
    N = df['recipe_id'].nunique()

    user_mapper = dict(zip(np.unique(df["user_id"]), list(range(M))))
    recipe_mapper = dict(zip(np.unique(df["recipe_id"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["user_id"])))
    recipe_inv_mapper = dict(zip(list(range(N)), np.unique(df["recipe_id"])))

    user_index = [user_mapper[i] for i in df['user_id']]
    item_index = [recipe_mapper[i] for i in df['recipe_id']]

    X = csr_matrix((df["rating"], (item_index,user_index)), shape=(N,M),dtype='uint8')

    return X, user_mapper, recipe_mapper, user_inv_mapper, recipe_inv_mapper

In [4]:
def item_based_rec(user, num_recommendation):
    number_neighbors = 3

    user_index = df.columns.tolist().index(user)

    for m, t in list(enumerate(df.index)):
        if df.iloc[m, user_index] == 0:
            sim_recipes = indices[m].tolist()
            recipes_distances = distances[m].tolist()

            if m in sim_recipes:
                id_recipe = sim_recipes.index(m)
                sim_recipes.remove(m)
                recipes_distances.pop(id_recipe)

            else:
                sim_recipes = sim_recipes[:3 - 1]
                recipes_distances = recipes_distances[:3 - 1]

            recipe_similarity = [1 - x for x in recipes_distances]
            recipe_similarity_copy = recipe_similarity.copy()
            nominator = 0

            for s in range(0, len(recipe_similarity)):
                if df.iloc[sim_recipes[s], user_index] == 0:
                    if len(recipe_similarity_copy) == (number_neighbors - 1):
                        recipe_similarity_copy.pop(s)

                    else:
                        recipe_similarity_copy.pop(s - (len(recipe_similarity) - len(recipe_similarity_copy)))

                else:
                    nominator = nominator + recipe_similarity[s] * df.iloc[sim_recipes[s], user_index]

            if len(recipe_similarity_copy) > 0:
                if sum(recipe_similarity_copy) > 0:
                    predicted_r = nominator / sum(recipe_similarity_copy)

                else:
                    predicted_r = 0

            else:
                predicted_r = 0

            df1.iloc[m, user_index] = predicted_r
    recommends(user, num_recommendation)

In [5]:
def recommends(user, num_recommended_recipes):
#     print('The list of the Recipes : user:{}  \n'.format(user_inv_mapper[user]))

#     for m in df[df[user] > 0][user].index.tolist():
#         rec=recipes["name"].loc[recipes['id']==recipe_inv_mapper[m]].to_string(index=False)
#         rat=sem[(sem['user_id']==(user_inv_mapper[user])) & (sem["recipe_id"]==recipe_inv_mapper[m])]
#         print(rec+":"+rat['rating'].to_string(index=False))
#         #print(recipe_inv_mapper[m])

    print('\n')

    recommended_recipes = []

    for m in df[df[user] == 0].index.tolist():
        index_df = df.index.tolist().index(m)
        predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
        recommended_recipes.append((m, predicted_rating))

    sorted_rm = sorted(recommended_recipes, key=lambda x: x[1], reverse=True)

    #print('The list of the Recommended Recipes \n')
    rank = 1
    for recommended_recipe in sorted_rm[:num_recommended_recipes]:
        my_rec.extend(list(recipes["name"].loc[recipes['id']==recipe_inv_mapper[recommended_recipe[0]]]))
        #if recommended_recipe[1]>=3:
        #print('{}: {} - predicted rating:{}'.format(rank, recipes["name"].loc[recipes['id']==recipe_inv_mapper[recommended_recipe[0]]].to_string(index=False), recommended_recipe[1]))
        #rank = rank + 1

In [51]:
my_rec=[]
sem=pd.read_csv('sem_an.csv')
recipes=pd.read_csv('RAW_recipes.csv')
sem=sem.groupby('recipe_id').filter(lambda x: len(x) >= 25 )
sem=sem.groupby('user_id').filter(lambda x: len(x) >= 3 )
print(sem.head(20))
X, user_mapper, recipe_mapper, user_inv_mapper, recipe_inv_mapper = create_X(sem)
df = pd.DataFrame(data=csr_matrix.todense(X))
print(df.shape)
df1=df.copy()
distances=pd.read_csv('distances.csv')
indices=pd.read_csv('indices.csv')
#print(distances.values)
distances=np.array(distances.values.tolist())
indices=np.array(indices.values.tolist())
#index=get_index(95810)



     user_id  recipe_id  rating
31     56680      79222       5
43    183565      79222       5
44    101823      79222       5
46    446143      79222       4
47    226989      79222       4
48    868654      79222       5
50    302867      79222       5
54    930021      79222       5
55    241697      79222       5
56    980899      79222       4
57    158966      79222       5
58    827374      79222       3
174   328099     195977       5
175   403193     195977       5
176    61995     195977       5
177   440735     195977       5
178   633027     195977       5
179   543968     195977       5
181  1701315     195977       5
188    91584     195977       5
(5834, 19828)


In [7]:
def col_fil(user_id):
    item_based_rec(user_mapper[user_id],10)
#col_fil(95810)

In [8]:
df_r=recipes.loc[recipes['id'].isin(sem['recipe_id'])]
df_r=df_r.reset_index(drop=True)


In [9]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df_r['description'] = df_r['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_r['description'])

#Output the shape of tfidf_matrix
#tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
def create_recs(user_id,df_r,sem):
    sem3=sem.loc[sem['user_id']== user_id]
    s=recipes.loc[recipes['id'].isin(sem3['recipe_id'])]
    s1=list(s['name'])
    #print(s1)
    recs=[]
    for i in s1:
        my_rec.extend(get_recommendations(user_id,i,df_r, cosine_sim))
    #return recs

In [33]:
def top_10():
    top_n = sem.groupby('recipe_id') \
           .agg({'user_id':'size', 'rating':'mean'}) \
           .rename(columns={'user_id':'count','rating':'mean_sent'}) \
           .reset_index()
    most_populary=top_n["recipe_id"][:10]
    most_populary
    print('Top 10 popular recipes:')
    for i in most_populary:
        print(list(recipes["name"].loc[recipes["id"]==i]))

In [47]:
def hybrid(user_id):
    top_10()
    create_recs(user_id,df_r,sem)
    rec=col_fil(user_id)
    print("We recommend you:")
    
   


In [48]:
hybrid(28649)

random.sample(my_rec,10)

Top 10 popular recipes:
['amish friendship bread and starter']
['classic cream scones']
['chinese hot and sour pork soup']
['lee s hot crab dip']
['spanish potato omelet  tortilla a la espanola']
['potatoes tapas in garlic mayonnaise  potatoes aioli']
['yellow squash puffs']
['blue mountain blueberry pie']
['remarkable rhubarb bites']
['lentil dal']


We recommend you:


['shirley s shrimp potato salad',
 'baked arroz con pollo',
 'delicious chicken pot pie',
 'mashed potatoes with creme fraiche and chives',
 'lasagna spirals',
 'amish chicken',
 'vegetarian taco soup',
 'southwest spaghetti squash',
 'cracker barrel green beans',
 'thanksgiving pumpkin pie  uses fresh pumpkin']

In [53]:
hybrid(95810)
random.sample(my_rec,10)


Top 10 popular recipes:
['amish friendship bread and starter']
['classic cream scones']
['chinese hot and sour pork soup']
['lee s hot crab dip']
['spanish potato omelet  tortilla a la espanola']
['potatoes tapas in garlic mayonnaise  potatoes aioli']
['yellow squash puffs']
['blue mountain blueberry pie']
['remarkable rhubarb bites']
['lentil dal']


We recommend you:


['crock pot apple or cherry cobbler',
 'the best  banana bread  or muffins',
 'rich and chewy low fat brownies',
 'the best  banana bread  or muffins',
 'turkey pot pie with stuffing crust  using leftovers',
 'light wheat beer bread',
 'my onion  fried  chicken',
 'parmesan acorn squash',
 'ultimate brownies',
 'irish pub beef stew']

In [50]:
hybrid(868654)
random.sample(my_rec,10)

Top 10 popular recipes:
['amish friendship bread and starter']
['classic cream scones']
['chinese hot and sour pork soup']
['lee s hot crab dip']
['spanish potato omelet  tortilla a la espanola']
['potatoes tapas in garlic mayonnaise  potatoes aioli']
['yellow squash puffs']
['blue mountain blueberry pie']
['remarkable rhubarb bites']
['lentil dal']


We recommend you:


['beef tips and rice',
 'parmesan acorn squash',
 'spinach stuffed turkey roll redone',
 'corn casserole  paula deen',
 'braised lamb shoulder chops',
 'heavenly potatoes and ham',
 'beef tips and rice',
 'broccoli casserole with no  cream of something  soups',
 'amish chicken',
 'one bowl brownie pie']

In [52]:
hybrid(868654)
random.sample(my_rec,10)

Top 10 popular recipes:
['amish friendship bread and starter']
['classic cream scones']
['chinese hot and sour pork soup']
['lee s hot crab dip']
['spanish potato omelet  tortilla a la espanola']
['potatoes tapas in garlic mayonnaise  potatoes aioli']
['yellow squash puffs']
['blue mountain blueberry pie']
['remarkable rhubarb bites']
['lentil dal']


We recommend you:


['oven roasted bbq chicken breast',
 'oven barbecued chicken wings',
 'easy stove top stuffing meatloaf',
 'mexican lime chicken',
 'greek cauliflower',
 'delicious cream of reuben soup',
 'mean chef s apple brine',
 'bread stuffing',
 'bailey s irish cream bread pudding',
 'old bay steak fries']

In [54]:
hybrid(302867)
random.sample(my_rec,10)

Top 10 popular recipes:
['amish friendship bread and starter']
['classic cream scones']
['chinese hot and sour pork soup']
['lee s hot crab dip']
['spanish potato omelet  tortilla a la espanola']
['potatoes tapas in garlic mayonnaise  potatoes aioli']
['yellow squash puffs']
['blue mountain blueberry pie']
['remarkable rhubarb bites']
['lentil dal']


We recommend you:


['easy lemon pound cake',
 'cheeseburger soup',
 'weight watchers parmesan chicken cutlets',
 'potatoes a l alsacienne',
 'easy crispy dijon chicken',
 'roasted tomato soup',
 'mashed potatoes with creme fraiche and chives',
 'look both ways snitch and run   potatoes',
 'chicken with laughing cow cheese',
 'sourdough coffee cake']